# NextRequest data cleaning and EDA
*Author: Steven Yuan*

This notebook contains potentially useful data cleaning and EDA routines for scraped NextRequest data.

In [4]:
import pandas as pd
import zipfile

In [5]:
test_df = pd.read_csv(zipfile.ZipFile('data/sd_requests.zip', 'r').open('sd_requests.csv'))
test_df.head()

,id,status,desc,date,depts,docs,poc,msgs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...","December 7, 2015 via web",Code Enforcement,"title,link\n5040 ShorehamPlace building permit...",Ginger Rodriguez,"title,item,time\n""Request Closed\nPublic"",02. ..."
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,"December 7, 2015 via web",Department of Real Estate and Airport Management,NaN,Jeffrey Wallace,"title,item,time\n""Request Published\nPublic"",,..."
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,"December 7, 2015 via web",City Clerk,"title,link\nhttp://www.sandiego.gov/park-and-r...",Mailei Ross-Cerezo,"title,item,time\n""Request Closed\nPublic"",Stil..."
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,"December 7, 2015 via web",Department of Real Estate and Airport Management,NaN,Jeffrey Wallace,"title,item,time\n""Request Closed\nPublic"",02. ..."
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...","December 7, 2015 via web",Code Enforcement,"title,link\nSite Plan - 11943 El Camino Real.p...",Ginger Rodriguez,"title,item,time\n""Request Closed\nPublic"",02. ..."


The following process converts the CSV strings in the `docs` and `msgs` columns into DataFrames:

In [3]:
df_fillna = lambda df: df.convert_dtypes().fillna('') if df is not None else None
test_df = df_fillna(test_df)
test_df

NameError: name 'test_df' is not defined

In [ ]:
csv_to_df = lambda csv: pd.read_csv(StringIO(csv)) if csv else None
test_df['docs_df'] = test_df['docs'].apply(csv_to_df)
test_df['msgs_df'] = test_df['msgs'].apply(csv_to_df)
test_df.head()

Then, we fill the NA values in the individual `docs` and `msgs` DataFrames:

In [ ]:
test_df['docs_df'] = test_df['docs_df'].apply(df_fillna)
test_df['msgs_df'] = test_df['msgs_df'].apply(df_fillna)
test_df.loc[4]['msgs_df']

Other EDA stuff:

In [ ]:
test_df.head()

In [ ]:
test_df.shape[0] # Number of requests scraped

In [ ]:
test_df[test_df['desc'].str.contains('Read more')] # Check if the descriptions were properly scraped

In [ ]:
empty_desc = test_df.query('desc == ""')
empty_desc

In [ ]:
# Check for empty depts field
empty_depts = test_df.query('depts == ""')
empty_depts

In [ ]:
# Check for empty docs field
empty_docs = test_df[test_df['docs'].str.fullmatch('title,link\n')]
empty_docs

In [ ]:
# Which requests had the longest message history? (Useful for finding worst-case scenarios for the scraper)
long_msgs = test_df['msgs_df'].apply(lambda df: df.shape[0] if df is not None else 0).sort_values(ascending=False)
long_msgs.head(10)

In [ ]:
# Sort requests by message history length
requests_long_msg = test_df.loc[long_msgs.index]
requests_long_msg.head(20)

In [ ]:
# Query for info about a specific request
request_id = '"17-3638"'
test_df.query('id == ' + request_id).iloc[0]['msgs_df']

In [ ]:
# Find request descriptions with the given substring, case insensitive
desc = 'Padres'
test_df[test_df['desc'].str.contains(desc, case=False)]

In [ ]:
# Find requests whose department(s) contain the given substring, case insensitive
dept = 'Police'
test_df[test_df['depts'].str.contains(dept, case=False)]

In [ ]:
# Convert empty dataframes from docs_df into None
remove_empty = lambda df: None if ((df is None) or (type(df) == str and not df) or df.empty) else df
test_df['docs_df'] = test_df['docs_df'].apply(remove_empty)
test_df[test_df['docs'].str.fullmatch('title,link\n')]

In [ ]:
# Split the date and request method from the date column
dates = test_df['date'].to_numpy()
test_df = test_df.join(
        pd.DataFrame(list(map(lambda x: x.split(' via '), dates)))
    ).drop(
        columns='date'
    ).rename(
        columns={0: 'date', 1: 'via'}
    ).convert_dtypes()
test_df.head()

In [ ]:
# Split the time and author from the time quote on each message
def split_time_author(msgs):
    if msgs is None:
        return None
    time_quotes = msgs['time'].to_numpy()
    time_author = pd.DataFrame(list(map(lambda x: x.split(' by '), time_quotes)))
    return df_fillna(msgs.join(
            time_author
        ).drop(
            columns='time'
        ).rename(
            columns={0: 'time', 1: 'by'}
        ))

test_df['msgs_df'] = test_df['msgs_df'].apply(split_time_author)
test_df.loc[4]['msgs_df']

In [ ]:
# Convert columns with time strings into DateTime
def convert_time_to_dt(df, col='time'):
    return df.assign(**{col + '_dt': pd.to_datetime(df[col])})

In [ ]:
# Splitting departments for easier pivoting
depts = test_df['depts'].to_numpy() # depts column
test_df_depts = test_df.join(pd.DataFrame(list(map(lambda x: x.split(', '), depts)))) # Split departments into separate columns
test_df_depts = test_df_depts.melt( # Melt on the individual departments
        id_vars=test_df.columns
    )[lambda df: df['value'].apply(lambda x: x is not None)].drop( # Get rid of None values
        columns='variable'
    ).rename( # Drop the variable column, rename the value column, and reset indices
        columns={'value': 'dept'}
    ).reset_index().drop(
        columns='index'
    )
test_df_depts

In [ ]:
test_df_depts.value_counts('dept')[lambda x: x.index.str.contains('Office', case=False)]

In [ ]:
# Find requests whose department(s) contain the given substring, case insensitive
dept = 'Chief Operating Officer'
test_df[test_df['depts'].str.contains(dept, case=False)]

In [ ]:
test_df = test_df.convert_dtypes()
test_df.loc[0]['msgs_df']

In [ ]:
test.loc[0]